# Исследование уровня потребительской лояльности телекоммуникационной компании
**Описание проекта**

Заказчик — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

Компания провела опрос и попросила подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite.

Чтобы оценить результаты опроса, оценки обычно делят на три группы:
9-10 баллов — «cторонники» (англ. promoters);
7-8 баллов — «нейтралы» (англ. passives);
0-6 баллов — «критики» (англ. detractors).

Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».

**Цель исследования**

1.  Построить дашборд, который представит информацию о текущем уровне NPS среди клиентов и покажет, как этот уровень меняется в зависимости от пользовательских признаков.

2.  Используя дашборд, ответьте на вопросы:

* Как распределены участники опроса по возрасту, полу и возрасту? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?

* Какие группы пользователей наиболее лояльны к сервису? Какие менее?

* Какой общий NPS среди всех опрошенных?

* Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?

3. Подготовить презентацию с ответами на вопросы.

### Подключение к базе и выгрузка нужных полей

In [1]:
# Импортируем библиотеки
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

In [2]:
# Подключаемся к БД SQLite
if os.path.exists('/datasets/telecomm_csi.db'):
    engine = create_engine(f'sqlite:////datasets/telecomm_csi.db', echo = False)
else:
    engine = create_engine(f'sqlite:///telecomm_csi.db', echo = False)

Предварительно посмотрев, что в полях age и gender_segment, имеются пропуски, с помощью оператора WHERE исключим их. 0,27% получим удаленных строк, это незначительный процент, на дальнейшее исследование не повлияет.

In [3]:
# Формируем sql-запрос, который выгрузит нужные нам поля.
query = """
SELECT u.user_id,
       u.lt_day,
       (CASE 
           WHEN u.lt_day < 365 THEN 'new client'
           ELSE 'old client'
       END) as is_new,
       u.age,
       (CASE 
           WHEN u.gender_segment = 1 THEN 'woman'
           ELSE 'man'
       END) as gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       SUBSTRING(ag.title, 4) as age_segment,
       SUBSTRING(ts.title, 4) as traffic_segment,
       SUBSTRING(ls.title,4) as lifetime_segment,
       u.nps_score,
       (CASE 
           WHEN u.nps_score >= 9 AND u.nps_score <= 10 THEN 'promoters'
           WHEN u.nps_score >= 7 AND u.nps_score <= 8 THEN 'passives'
           ELSE 'detractors'
       END) as nps_group
FROM user as u
JOIN location as l ON u.location_id=l.location_id
JOIN age_segment as ag ON u.age_gr_id=ag.age_gr_id
JOIN traffic_segment ts ON u.tr_gr_id=ts.tr_gr_id
JOIN lifetime_segment ls ON u.lt_gr_id=ls.lt_gr_id
WHERE u.age AND u.gender_segment IS NOT NULL;
"""

In [4]:
# Выполняем запрос, смотрим первые 10 строк и сохраняем результат выполнения в DataFrame.
df = pd.read_sql(query, engine)
df.head(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,old client,45.0,woman,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,promoters
1,A001WF,2344,old client,53.0,man,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,promoters
2,A003Q7,467,old client,57.0,man,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,promoters
3,A004TB,4190,old client,44.0,woman,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,promoters
4,A004XT,1163,old client,24.0,man,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,promoters
5,A005O0,5501,old client,42.0,woman,ANDROID,SMARTPHONE,Россия,Омск,35-44,5-10,36+,6,detractors
6,A0061R,1236,old client,45.0,man,ANDROID,SMARTPHONE,Россия,Уфа,45-54,10-15,36+,10,promoters
7,A009KS,313,new client,35.0,man,ANDROID,SMARTPHONE,Россия,Москва,35-44,45-50,7-12,10,promoters
8,A00AES,3238,old client,36.0,woman,ANDROID,SMARTPHONE,Россия,СанктПетербург,35-44,1-5,36+,10,promoters
9,A00F70,4479,old client,54.0,woman,ANDROID,SMARTPHONE,Россия,Волгоград,45-54,15-20,36+,9,promoters


In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

In [6]:
# изучим тип данных у полей
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501152 entries, 0 to 501151
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           501152 non-null  object 
 1   lt_day            501152 non-null  int64  
 2   is_new            501152 non-null  object 
 3   age               501152 non-null  float64
 4   gender_segment    501152 non-null  object 
 5   os_name           501152 non-null  object 
 6   cpe_type_name     501152 non-null  object 
 7   country           501152 non-null  object 
 8   city              501152 non-null  object 
 9   age_segment       501152 non-null  object 
 10  traffic_segment   501152 non-null  object 
 11  lifetime_segment  501152 non-null  object 
 12  nps_score         501152 non-null  int64  
 13  nps_group         501152 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.5+ MB


### Ссылка на дашборд на сайте Tableau Public:

<https://public.tableau.com/authoring/project_7_16500531851140/Dashboard1#2>

### Ссылка на pdf-файл с презентацией:

<https://disk.yandex.ru/i/WpSjrLQP0gk7pQ>